!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!This script contains maskign code before standardisation. however standardisation of inputs and targets was performed before the interpoléated baselines were generated via CDO for bilinear and bicubic and then masked so that they only retain the Swiss Domain

In [13]:
from dependencies import *
import sys
sys.path.append("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/Scripts/Functions")

In [14]:
from dask.distributed import Client
client= Client()
print(client)

/users/sasthana/.local/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41793 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:40959' processes=4 threads=4, memory=503.36 GiB>


bilinear interpolation to the 1 km grid was performed using CDO remapbil with RhiresD/TabsD as reference files. Same for remapbic (Bicubic interpolation) Now they will be masked in this script so that it only retains the Swiss domain and not entire Europe

#For precipitation mask

In [15]:
hr_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc"
mask_output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_mask.nc"

ds_hr = xr.open_dataset(hr_path)

varname = "RhiresD" 

mask_rhiresd = xr.where(~np.isnan(ds_hr[varname].isel(time=0)), 1, 0)

mask_rhiresd = mask_rhiresd.squeeze(drop=True)

mask_rhiresd.to_netcdf(mask_output_path)

print(f"Mask saved at {mask_output_path}")


Mask saved at /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_mask.nc


#For temperature mask!

In [16]:
hr_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc"
mask_output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_mask.nc"

ds_hr = xr.open_dataset(hr_path)

varname = "TabsD" 

mask_tabsd = xr.where(~np.isnan(ds_hr[varname].isel(time=0)), 1, 0)

mask_tabsd = mask_tabsd.squeeze(drop=True)



In [17]:
mask_tabsd.to_netcdf(mask_output_path)

print(f"Mask saved at {mask_output_path}")

Mask saved at /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_mask.nc


#Application of the mask to the two datasets : RhiresD and TabsD masks (1km) to the bicubically interpolated 1 km datasets features from Sven`s files so that it only retains the Swiss Domain


In [19]:
bicubic_ds = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/features_bicubic_precip_1971_2022_non_masked.nc",chunks={"time": 500})

# Masking
masked_bicubic_rhiresd = bicubic_ds * mask_rhiresd

masked_bicubic_rhiresd.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/precip_1km_bicubic_Swiss_features_masked.nc",
                                 engine="netcdf4",
                                    compute=True)


#Compute time of the above cell : 1 hour!!!!!!!!!!
#Could have used h5netcdf for faster computation
#Could have asked for more resources installed dask distributed, and used all 4 CPU cores, computing time reduced to 15 seconds!!!

In [20]:
bilinear_ds = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/features_bilinear_precip_1971_2022_non_masked.nc",chunks={"time": 500})

# Masking
masked_bilinear_rhiresd = bilinear_ds * mask_rhiresd

masked_bilinear_rhiresd.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/precip_1km_bilinear_Swiss_features_masked.nc",
                                 engine="netcdf4",
                                    compute=True)

Lazy loading was used below to allow for parallel computation. The data was not loading due to xarray loading everything at once and the kernel was crashing

In [21]:
bicubic_ds_tabsd = xr.open_dataset(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/features_bicubic_tas_1971_2022_non_masked.nc",
    chunks={"time": 500} #Taking 100 days at a time
)
#Masking
masked_bicubic_tabsd = bicubic_ds_tabsd * mask_tabsd

masked_bicubic_tabsd.to_netcdf(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/tas_1km_bicubic_Swiss_features_masked.nc",
    engine="netcdf4",
    compute=True
)

In [22]:
bilinear_ds_tabsd = xr.open_dataset(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/features_bilinear_tas_1971_2022_non_masked.nc",
    chunks={"time": 500} #Taking 100 days at a time
)
#Masking
masked_bilinear_tabsd = bilinear_ds_tabsd * mask_tabsd

masked_bilinear_tabsd.to_netcdf(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/tas_1km_bilinear_Swiss_features_masked.nc",
    engine="netcdf4",
    compute=True
)

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxCHECKING SHAPExxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [23]:

features_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/precip_1km_bicubic_Swiss_features_masked.nc")["pr"]

In [24]:
features_ds_temp= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/tas_1km_bicubic_Swiss_features_masked.nc")["tas"]

In [25]:
features_ds_precip.shape

(18993, 265, 370)

In [26]:
features_ds_temp.shape

(18993, 265, 370)

In [27]:
targets_precip= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc")["RhiresD"]
targets_precip.shape

(18993, 265, 370)

In [28]:
targets_temp= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc")["TabsD"]
targets_temp.shape

(18993, 265, 370)

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxStandardisation code (please note that the above propcessed files have already been standardised before performing the interpilation and masking propcedures) xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

#Standardising the dataset depending on the variable for feeding into the network

In [5]:
from Standardise import standardise

In [5]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_scaled_1971_2022.nc'
var = 'TabsD'

standardise(input_path, output_path, var)


/users/sasthana/.local/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_scaled_1971_2022.nc


In [6]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_scaled_1971_2022.nc'
var = 'RhiresD'

standardise(input_path, output_path, var)

Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_scaled_1971_2022.nc


In [3]:
#standardiigng features as well

In [6]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/tas_daily_1971_2022_cropped.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/tas_daily_cropped_scaled_1971_2022.nc'
var = 'tas'

standardise(input_path, output_path, var)

/users/sasthana/.local/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/tas_daily_cropped_scaled_1971_2022.nc


In [7]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/precip_daily_1971_2022_cropped.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/precip_daily_cropped_scaled_1971_2022.nc'
var = 'pr'

standardise(input_path, output_path, var)

Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/precip_daily_cropped_scaled_1971_2022.nc
